<a href="https://colab.research.google.com/github/GavinHacker/ml_practice/blob/master/recbycontent_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pymysql

In [2]:
!pip install git+https://github.com/coreylynch/pyFM

  Cloning https://github.com/coreylynch/pyFM to /tmp/pip-req-build-22dr07g_
  Running setup.py bdist_wheel for pyfm ... - \ | / - done
  Stored in directory: /tmp/pip-ephem-wheel-cache-mq3vt_t1/wheels/3b/d9/ef/1b148c527d39344632833679e79b3db1798a40b0f64f917b13
Successfully built pyfm


In [0]:
import pandas as pd
import pymysql
import pymysql.cursors
from functools import reduce
import numpy as np
import pandas as pd
import uuid
import datetime
from pyfm import pylibfm
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics.pairwise import pairwise_distances
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

In [0]:
def get_connection():
    return pymysql.connect(host='rm-2zeqqm6994abi7b6dqo.mysql.rds.aliyuncs.com',
                               user='noone',
                               password='Huawei12#$',
                               db='recsys',
                               port=3306,
                               charset ='utf8',
                               use_unicode=True)

In [0]:
df = pd.read_sql_query("select * from movie", get_connection())

In [6]:
df.shape

(6629, 10)

In [0]:
df_exp = df.head(1)

In [8]:
df_exp

id    name             ADD_TIME         type  \
0  10001432  颠倒的帕特玛  2018-02-07 20:46:08  剧情|冒险|科幻|爱情   

                      actors region director trait  rat enable  
0  藤井雪代|冈本信彦|大畑伸太郎|福松进纱|加藤将之     日本     吉浦康裕    青春  7.9      1

In [0]:
def get_dim_dict(df, dim_name):
  type_list = list(map(lambda x:x.split('|') ,df[dim_name]))
  type_list = [x for l in type_list for x in l]
  def reduce_func(x, y):
    for i in x:
      if i[0] == y[0][0]:
        x.remove(i)
        x.append(((i[0],i[1] + 1)))
        return x
    x.append(y[0])
    return x
  l = filter(lambda x:x != None, map(lambda x:[(x, 1)], type_list))
  type_zip = reduce(reduce_func, list(l))
  #type_list = sorted(list(set(type_list)))
  #type_zip = zip(list(range(len(type_list))), type_list)
  print(len(type_zip))
  print(type_zip)
  type_dict = {}
  for i in type_zip:
    type_dict[i[0]] = i[1]
  return type_dict

In [10]:
type_dict = get_dim_dict(df, 'type')
actors_dict = get_dim_dict(df, 'actors')
director_dict = get_dim_dict(df, 'director')
trait_dict = get_dim_dict(df, 'trait')

21
[('西部', 54), ('同性', 318), ('恐怖', 463), ('情色', 289), ('战争', 447), ('灾难', 114), ('音乐', 425), ('歌舞', 225), ('奇幻', 1099), ('传记', 478), ('悬疑', 1027), ('科幻', 817), ('武侠', 150), ('历史', 466), ('爱情', 3312), ('惊悚', 1172), ('剧情', 5191), ('犯罪', 1350), ('喜剧', 2856), ('冒险', 974), ('动作', 1661)]
12558
[('藤井雪代', 1), ('大畑伸太郎', 1), ('福松进纱', 1), ('加藤将之', 1), ('丹尼·特雷霍', 1), ('兰比尔·卡普尔', 1), ('伊利亚娜·狄克鲁兹', 1), ('Akash Khurana', 1), ('Karlheinz Hackl', 1), ('Wolfgang M', 1), ('Bauer', 1), ('Ulrike Beimpold', 1), ('Julia Gschnitzer', 1), ('朴敏荷', 1), ('李熙俊', 1), ('吴坚', 1), ('李佳蔚', 1), ('祖锋', 1), ('萨尔瓦多·斯特里亚诺', 1), ('柯西莫·雷加', 1), ('安东尼奥·弗拉斯加', 1), ('胡安·达里奥·伯内蒂', 1), ('文森特·盖洛', 1), ('Adam Savage', 1), ('Jamie Hyneman', 1), ('卡文·英玛诺泰', 1), ('露丝·莱斯利', 1), ('朱莉·恩格尔布雷希特', 1), ('艾玛·福尔曼', 1), ('妮珂莱特·皮尔妮 ', 1), ('姜峰', 1), ('尼玛颂宋', 1), ('雷茜', 1), ('游大庆', 1), ('米洛·文堤米利亚', 1), ('宫藤官九郎', 1), ('胡夏', 1), ('大左', 1), ('徐正超', 1), ('迈克尔·C·豪尔', 1), ('黛西·贝文', 1), ('大卫·沃肖夫斯基', 1), ('张岫云', 1), ('陈文彬', 1), ('罗贝尔·普拉尼奥尔', 1), ('乔纳森·科恩

In [11]:
#print(type_dict)
print(actors_dict)
print(director_dict)
#print(trait_dict)

{'藤井雪代': 1, '大畑伸太郎': 1, '福松进纱': 1, '加藤将之': 1, '丹尼·特雷霍': 1, '兰比尔·卡普尔': 1, '伊利亚娜·狄克鲁兹': 1, 'Akash Khurana': 1, 'Karlheinz Hackl': 1, 'Wolfgang M': 1, 'Bauer': 1, 'Ulrike Beimpold': 1, 'Julia Gschnitzer': 1, '朴敏荷': 1, '李熙俊': 1, '吴坚': 1, '李佳蔚': 1, '祖锋': 1, '萨尔瓦多·斯特里亚诺': 1, '柯西莫·雷加': 1, '安东尼奥·弗拉斯加': 1, '胡安·达里奥·伯内蒂': 1, '文森特·盖洛': 1, 'Adam Savage': 1, 'Jamie Hyneman': 1, '卡文·英玛诺泰': 1, '露丝·莱斯利': 1, '朱莉·恩格尔布雷希特': 1, '艾玛·福尔曼': 1, '妮珂莱特·皮尔妮 ': 1, '姜峰': 1, '尼玛颂宋': 1, '雷茜': 1, '游大庆': 1, '米洛·文堤米利亚': 1, '宫藤官九郎': 1, '胡夏': 1, '大左': 1, '徐正超': 1, '迈克尔·C·豪尔': 1, '黛西·贝文': 1, '大卫·沃肖夫斯基': 1, '张岫云': 1, '陈文彬': 1, '罗贝尔·普拉尼奥尔': 1, '乔纳森·科恩': 1, '薇迪亚·巴兰': 1, '帕拉姆巴拉塔·查特吉': 1, '特里蒂曼·查特吉': 1, '萨斯瓦塔·查特吉': 1, '丁佳明': 1, '简嫚书': 1, '吴卓羲': 1, '孙祖扬': 1, '尹雅宁': 1, '吴冰滨': 1, '赤羽根健治': 1, '李贤镇': 1, '李素晶': 1, '李茂生': 1, '托尼·丹扎': 1, '林·沙烨': 1, '赵敏修': 1, '伍基洪': 1, '姜恩珍': 1, 'Anna Chuancheun': 1, '曹议文': 1, '葛蕾': 1, '阿加塔·布泽克': 1, '盖·莱恩': 1, '朴晓英': 1, '袁牧女': 1, '王烈': 1, '卡里·希耶塔拉赫蒂': 1, '沃科·霍瓦塔': 1, '尤哈·韦约宁': 1, '埃里克·卡尔森': 1, '艾丽娜·克尼赫蒂

In [12]:
director_dict['吉姆·弗尔']

2

In [13]:
actors_dict['马克·莱昂纳蒂']

2

In [0]:
df_ = df.drop(['ADD_TIME', 'enable', 'rat', 'id', 'name'], axis=1)

In [15]:
df_.head()

type                                            actors  \
0              剧情|冒险|科幻|爱情                         藤井雪代|冈本信彦|大畑伸太郎|福松进纱|加藤将之   
1  奇幻|惊悚|爱情|战争|剧情|科幻|冒险|动作          詹妮弗·劳伦斯|乔什·哈切森|利亚姆·海姆斯沃斯|伍迪·哈里森|伊丽莎白·班克斯   
2              喜剧|犯罪|惊悚|动作        丹尼·特雷霍|米歇尔·罗德里格兹|索菲娅·维加拉|艾梅柏·希尔德|Lady Gaga   
3              剧情|喜剧|音乐|爱情  佩丽冉卡·曹帕拉|兰比尔·卡普尔|伊利亚娜·狄克鲁兹|Akash Khurana|沙鲁巴·舒克拉   
4                    剧情|喜剧              扎克瑞·戈登|瑞秋·哈里斯|戴文·博斯蒂克|罗伯特·卡普荣|佩顿·李斯特   

  region   director           trait  
0     日本       吉浦康裕              青春  
1     美国   弗朗西斯·劳伦斯              青春  
2    俄罗斯  罗伯特·罗德里格兹              搞笑  
3     印度    阿努拉格·巴苏  感人|经典|励志|搞笑|文艺  
4    加拿大     大卫·博维斯           青春|搞笑

In [0]:
movie_dict_list = []
for i in df_.index:
  movie_dict = {}
  #type
  for s_type in df_.iloc[i]['type'].split('|'):
    movie_dict[s_type] = 1
  #actors
  for s_actor in df_.iloc[i]['actors'].split('|'):
    if actors_dict[s_actor] < 2:
      movie_dict['other_actor'] = 1
    else:
      movie_dict[s_actor] = 1
  #regios
  movie_dict[df_.iloc[i]['region']] = 1
  #director
  for s_director in df_.iloc[i]['director'].split('|'):
    if director_dict[s_director] < 2:
      movie_dict['other_director'] = 1
    else:
      movie_dict[s_director] = 1
  #trait
  for s_trait in df_.iloc[i]['trait'].split('|'):
    movie_dict[s_trait] = 1
  movie_dict_list.append(movie_dict)

In [0]:
v = DictVectorizer()
X = v.fit_transform(movie_dict_list)

In [18]:
X.shape

(6629, 5855)

In [19]:
pd.DataFrame(data=X.toarray(), columns=v.feature_names_).T.head(5)

0     1     2     3     4     5     6     7     8     \
50分                    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
Aaron Seltzer          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
Aleksandr Voytinskiy   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
Allan Brocka           0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
Andrei Shchetinin      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

                      9     ...   6619  6620  6621  6622  6623  6624  6625  \
50分                    0.0  ...    0.0   0.0   0.0   0.0   0.0   0.0   0.0   
Aaron Seltzer          0.0  ...    0.0   0.0   0.0   0.0   0.0   0.0   0.0   
Aleksandr Voytinskiy   0.0  ...    0.0   0.0   0.0   0.0   0.0   0.0   0.0   
Allan Brocka           0.0  ...    0.0   0.0   0.0   0.0   0.0   0.0   0.0   
Andrei Shchetinin      0.0  ...    0.0   0.0   0.0   0.0   0.0   0.0   0.0   

                      6626  6627  6628  
50分                    0.0   0.0   0.0  
Aaron Seltzer          0.0   0.0   0.0  
Aleksandr Voytinskiy   0.0   0.0   0.0  
Allan Brocka           0.0   0.0   0.0  
Andrei Shchetinin      0.0   0.0   0.0  

[5 rows x 6629 columns]

In [0]:
item_similarity = pairwise_distances(X, metric='cosine')

In [21]:
item_similarity.shape

(6629, 6629)

In [22]:
compare_index = 3
index = 0
_max_index = 0
_max = 1
for i in item_similarity[compare_index]:
  if i < _max and i != 0:
    _max = i
    _max_index = index
  index = index + 1
print(_max_index, _max)

3634 0.2587506833388987


In [0]:
index_of_sim = _max_index

In [24]:
df.iloc[index_of_sim]
movie_dict_list[index_of_sim]
df_106 = pd.DataFrame(data=X.todense()[index_of_sim], columns=v.feature_names_)
df_0 = pd.DataFrame(data=X.todense()[compare_index], columns=v.feature_names_)
df_diff = pd.concat([df_0, df_106], axis=0, ignore_index=True)
#df_diff.reindex(['0','1'])
df_diff = df_diff.T
df_diff[(df_diff[0] != 0) | (df_diff[1] != 0)]

0    1
other_actor     1.0  1.0
other_director  1.0  1.0
丹尼·博伊尔          0.0  1.0
佩丽冉卡·曹帕拉        1.0  0.0
剧情              1.0  1.0
励志              1.0  1.0
印度              1.0  1.0
喜剧              1.0  1.0
感人              1.0  1.0
戴夫·帕特尔          0.0  1.0
搞笑              1.0  0.0
文艺              1.0  0.0
沙鲁巴·舒克拉         1.0  1.0
爱情              1.0  1.0
经典              1.0  1.0
青春              0.0  1.0
音乐              1.0  0.0

In [25]:
df.iloc[index_of_sim]
#movie_dict_list[0]

id                                          2209573
name                                       贫民窟的百万富翁
ADD_TIME                   Fri Jan  5 11:35:05 2018
type                                       剧情|喜剧|爱情
actors      戴夫·帕特尔|沙鲁巴·舒克拉|亚尼·卡普|拉詹德拉纳斯·祖特施|吉尼瓦·塔瓦尔
region                                           印度
director                              丹尼·博伊尔|洛芙琳·坦丹
trait                                   感人|青春|经典|励志
rat                                             8.5
enable                                            1
Name: 3634, dtype: object

In [26]:
df.iloc[compare_index]

id                                                  10355329
name                                                 巴菲的奇妙命运
ADD_TIME                                 2018-02-07 20:44:01
type                                             剧情|喜剧|音乐|爱情
actors      佩丽冉卡·曹帕拉|兰比尔·卡普尔|伊利亚娜·狄克鲁兹|Akash Khurana|沙鲁巴·舒克拉
region                                                    印度
director                                             阿努拉格·巴苏
trait                                         感人|经典|励志|搞笑|文艺
rat                                                      8.3
enable                                                     1
Name: 3, dtype: object

In [27]:
item_similarity.shape

(6629, 6629)

In [28]:
li_dict = {1:0.1, 2:0.2, 5:0.4, 8:0.3}
min(li_dict.values())
dict(filter(lambda x:x[1] != 0.4 ,li_dict.items()))

{1: 0.1, 2: 0.2, 8: 0.3}

In [0]:
df_sim = pd.DataFrame(data=item_similarity)

In [30]:
df_sim.head(1)

0         1         2         3         4         5       6         7     \
0   0.0  0.583333  0.903775  0.643652  0.798992  0.555556  0.6302  0.842865   

       8        9       ...         6619    6620      6621      6622  \
0  0.701858  0.72265    ...     0.367544  0.6302  0.622036  0.698489   

       6623     6624    6625      6626      6627      6628  
0  0.396977  0.80755  0.8151  0.863917  0.711325  0.821826  

[1 rows x 6629 columns]

In [0]:
def insert_one_ibmovie(id_,
                      movieid,
                      recmovieid,
                      recmovierat,
                      simrat,
                      time,
                      enable, connection):
  sql = 'insert into ibmovie (id, movieid, recmovieid, recmovierat, simrat, time, enable) values (\'%s\',\'%s\',\'%s\',\'%s\',\'%s\',\'%s\',\'%s\')' % (id_, movieid, recmovieid, recmovierat, simrat, time, enable)
  print(sql)
  try:
    with connection.cursor() as cursor:
        cout=cursor.execute(sql)
    connection.commit()
  except Exception as e:
    print(e)
    connection.close()

## Test code

In [32]:
_conn = get_connection()
insert_one_ibmovie(uuid.uuid4(), '1', '2', '6', '0.1', datetime.datetime.now(), '1', _conn)
_conn.commit()
_conn.close()

insert into ibmovie (id, movieid, recmovieid, recmovierat, simrat, time, enable) values ('9cd233fb-bb9f-49f3-8984-29a007924cea','1','2','6','0.1','2018-12-20 08:48:51.919404','1')


In [33]:
df_sim.nsmallest(10, 0)[0]

0       0.000000
4543    0.222222
3848    0.244071
4595    0.244071
5123    0.244071
5780    0.244071
6381    0.262135
2354    0.292893
4064    0.292893
5176    0.292893
Name: 0, dtype: float64

In [0]:
def process_offline_compute_by_cosdis(rec_per_num):
  rec_per_num = rec_per_num + 1
  connection = get_connection()
  for i in range(0, item_similarity.shape[0]):
    df_sim_p = df_sim.nsmallest(rec_per_num, i)
    df_sim_p = df_sim_p[i]
    movie_id = df.iloc[i]['id']
    time_now = datetime.datetime.now()
    for rec_movie_item in df_sim_p.to_dict().items():
      if rec_movie_item[0] != i:
        rec_movie_index = rec_movie_item[0]
        rec_movie_sim = rec_movie_item[1]
        rec_movie_id = df.iloc[rec_movie_index]['id']
        rec_movie_rat = df.iloc[rec_movie_index]['rat']
        insert_one_ibmovie(id_=uuid.uuid4(), movieid=movie_id, recmovieid=rec_movie_id, recmovierat=rec_movie_rat, simrat=rec_movie_sim, time=time_now, enable='1', connection=connection)
    connection.commit()

In [50]:
process_offline_compute_by_cosdis(rec_per_num=2)

insert into ibmovie (id, movieid, recmovieid, recmovierat, simrat, time, enable) values ('2ea27d62-f4c0-4e50-9de4-bcb0427eeb63','10001432','26375689','6.8','0.2222222222222221','2018-12-20 09:01:36.147226','1')
insert into ibmovie (id, movieid, recmovieid, recmovierat, simrat, time, enable) values ('eba4c82f-24d3-4705-aaca-2d2309c83a8b','10001432','2381835','5.7','0.24407105398154572','2018-12-20 09:01:36.147226','1')
insert into ibmovie (id, movieid, recmovieid, recmovierat, simrat, time, enable) values ('647c3607-8cb2-4aae-8271-e1de0b5f35ab','10047547','6874456','7.3','0.03175416344814619','2018-12-20 09:01:38.639798','1')
insert into ibmovie (id, movieid, recmovieid, recmovierat, simrat, time, enable) values ('7f535096-fe13-4ff6-aaba-a5802d3511c5','10047547','6533054','5.9','0.09630388588493644','2018-12-20 09:01:38.639798','1')
insert into ibmovie (id, movieid, recmovieid, recmovierat, simrat, time, enable) values ('6fa5b62f-dc5d-46cf-8da5-7638355b7489','10342838','1305858','8.0','

KeyboardInterrupt: ignored

In [0]:
df_sim_3_dict = df_sim_3[4].to_dict()
for i in df_sim_3_dict.items():
  print(i)

(3, 0.7582531107923859)
(3634, 0.7491273969978728)
(223, 0.7272727272727273)
(4786, 0.6363636363636364)
(6115, 0.6363636363636364)
(5966, 0.6776708143898479)
(3111, 0.6984886554222364)
(3263, 0.6984886554222364)
(3723, 0.7989924369481576)
(4654, 0.7989924369481576)


In [0]:
print(df.iloc[3263]['id'])
print(df.iloc[3263]['rat'])


20254365
7.5
